In [19]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import datetime

In [20]:
df = pd.read_csv('Diesel.csv')

In [21]:
df

,Location,Month,Prices
0,"Charlottetown, Prince Edward Island",Jan-90,50.3
1,"Charlottetown, Prince Edward Island",Feb-90,50.5
2,"Charlottetown, Prince Edward Island",Mar-90,50.8
3,"Charlottetown, Prince Edward Island",Apr-90,51.1
4,"Charlottetown, Prince Edward Island",May-90,51.7
...,...,...,...
6947,"Yellowknife, Northwest Territories",Aug-23,175.7
6948,"Yellowknife, Northwest Territories",Sep-23,180.3
6949,"Yellowknife, Northwest Territories",Oct-23,183.1
6950,"Yellowknife, Northwest Territories",Nov-23,187.3


In [22]:
df['Month'] = pd.to_datetime(df['Month'], format='%b-%y')

In [23]:
df['Year'] = df['Month'].dt.year
df['Month_Num'] = df['Month'].dt.month

In [24]:
df = df[pd.to_numeric(df['Prices'], errors='coerce').notnull()]

In [25]:
df['Prices'] = df['Prices'].astype(float)

C:\Users\jack\AppData\Local\Temp\ipykernel_23860\512960355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Prices'] = df['Prices'].astype(float)


In [26]:
groups = df.groupby('Location')
df

,Location,Month,Prices,Year,Month_Num
0,"Charlottetown, Prince Edward Island",1990-01-01,50.3,1990.0,1.0
1,"Charlottetown, Prince Edward Island",1990-02-01,50.5,1990.0,2.0
2,"Charlottetown, Prince Edward Island",1990-03-01,50.8,1990.0,3.0
3,"Charlottetown, Prince Edward Island",1990-04-01,51.1,1990.0,4.0
4,"Charlottetown, Prince Edward Island",1990-05-01,51.7,1990.0,5.0
...,...,...,...,...,...
6947,"Yellowknife, Northwest Territories",2023-08-01,175.7,2023.0,8.0
6948,"Yellowknife, Northwest Territories",2023-09-01,180.3,2023.0,9.0
6949,"Yellowknife, Northwest Territories",2023-10-01,183.1,2023.0,10.0
6950,"Yellowknife, Northwest Territories",2023-11-01,187.3,2023.0,11.0


In [27]:
predicted_df = pd.DataFrame()


""


In [33]:
def predict_for_group(group):  # Define a function for clarity
    # Splitting the data into features (X) and target variable (y)
    X = group[['Year', 'Month_Num']]
    y = group['Prices']

    # Training a linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Create a DataFrame for future dates for this location
    max_date = group['Month'].dt.to_period('M').max().to_timestamp('M')  
    next_month_start = max_date + pd.offsets.MonthBegin(1)  
    future_dates = pd.date_range(start=next_month_start, periods=60, freq='M')  

    future_df = pd.DataFrame({'Month': future_dates})
    future_df['Year'] = future_df['Month'].dt.year
    future_df['Month_Num'] = future_df['Month'].dt.month

    # Make predictions for this location
    future_predictions = model.predict(future_df[['Year', 'Month_Num']])

    # Add predictions to the predicted DataFrame
    future_df['Predicted Prices'] = future_predictions
    future_df['Location'] = name

    return future_df  # Return the predictions for this group

# Apply changes to all groups:
predicted_df = pd.DataFrame()  # Initialize empty DataFrame
for name, group in groups:
    predictions = predict_for_group(group)
    predicted_df = pd.concat([predicted_df, predictions])


In [35]:
# Round predicted prices to 1 decimal place
predicted_df['Predicted Prices'] = predicted_df['Predicted Prices'].round(1)
predicted_df['Month'] = predicted_df['Month_Num']
predicted_df.drop('Month_Num', axis=1, inplace=True)
predicted_df.reset_index(drop=True, inplace=True)
predicted_df.head(12)


,Month,Year,Predicted Prices,Location
0,1,2024,140.0,"Calgary, Alberta"
1,2,2024,140.5,"Calgary, Alberta"
2,3,2024,140.9,"Calgary, Alberta"
3,4,2024,141.4,"Calgary, Alberta"
4,5,2024,141.8,"Calgary, Alberta"
5,6,2024,142.2,"Calgary, Alberta"
6,7,2024,142.7,"Calgary, Alberta"
7,8,2024,143.1,"Calgary, Alberta"
8,9,2024,143.6,"Calgary, Alberta"
9,10,2024,144.0,"Calgary, Alberta"


In [36]:
predicted_df.to_csv('Predicted_Diesel_Prices.csv', index=False)